In [ ]:
%run "0b_Create_poi_files.ipynb"

## Map HRU's, segments and POIs as folium.map elements

#### Set mapping extent and base map elements


In [ ]:
# Set approximate latitude, longitude and zoom level for subbasin is calculated for starting point of folium.map plot window.
pfile_lat = hru_gdf["hru_lat"].mean()
pfile_lon = hru_gdf["hru_lon"].mean()
zoom = 7  # Can be ad

In [ ]:
# Set up a custom tile for base map

# This can be tricky with syntax but if you go to this link you will find resources that have options beyond the few defualt options in folium leaflet.
# http://leaflet-extras.github.io/leaflet-providers/preview/
# These tiles will also work in the minimap, but can get glitchy if the same tile var is used in the minimap and the main map child object.

USGStopo_layer = folium.TileLayer(
    tiles="https://basemap.nationalmap.gov/arcgis/rest/services/USGSTopo/MapServer/tile/{z}/{y}/{x}",
    attr="USGS_topo",
    zoom_start=zoom,
    name="USGSTopo",
)
USGSHydroCached_layer = folium.TileLayer(
    tiles="https://basemap.nationalmap.gov/arcgis/rest/services/USGSHydroCached/MapServer/tile/{z}/{y}/{x}",
    attr="USGSHydroCached",
    zoom_start=zoom,
    name="USGSHydroCached",
)

In [ ]:
# Set up inset map
# This requires folium plugins. (from folium import plugins)
minimap = plugins.MiniMap(
    tile_layer="OpenStreetMap",
    # attr = 'USGS_topo',
    position="topleft",
    # zoom_level_offset=- 4,
    height=200,
    width=200,
    collapsed_height=25,
    collapsed_width=25,
    zoom_level_fixed=5,
    toggle_display=True,
    # collapsed = True
)

In [ ]:
crs = hru_gdf.crs

## Create interactive HRU map

### Create new HRU folium.map

In [ ]:
# Style functions
style_function_hru_map = lambda x: {
    "opacity": 1,
    "fillColor": "#00000000",  #'goldenrod',
    "color": "tan",
    "weight": 1.5,
}
highlight_function_hru_map = lambda x: {
    "opacity": 0.5,
    "color": "gray",
    "fillColor": "gray",
    "weight": 3,
}
style_function_seg_map = lambda x: {"opacity": 1, "color": "#217de7", "weight": 2}
highlight_function_seg_map = lambda x: {"opacity": 1, "color": "white", "weight": 4}
transparent = lambda x: {
    "fillColor": "#00000000",
    "color": "#00000000",
    "weight": 4,
}

In [ ]:
from folium.features import DivIcon

m2 = folium.Map(
    location=[pfile_lat, pfile_lon],
    tiles=USGSHydroCached_layer,
    zoom_start=zoom,
    width="100%",
    height="100%",
    control_scale=True,
)

folium.TileLayer(
    tiles="https://basemap.nationalmap.gov/arcgis/rest/services/USGSTopo/MapServer/tile/{z}/{y}/{x}",
    attr="USGS_Topo",
    zoom_start=zoom,
    name="USGS Topography",
    show=False,
).add_to(m2)

#############################################################################################################################################################
tooltip_hru = folium.GeoJsonTooltip(
    fields=["nhm_id", "hru_segment_nhm"],
    aliases=["hru", " flows to segment"],
    labels=True,
    # style=("background-color: #F0EFEF;border: 2px solid black;font-family: arial; font-size: 16px; padding: 10px;"),# Note that this tooltip style sets the style for all tool_tips.
)

popup_hru = folium.GeoJsonPopup(
    fields=["nhm_id", "hru_segment_nhm"],
    aliases=["hru", " flows to segment"],
    labels=True,
    localize=False,
    style=(
        "font-size: 16px;"
    ),  # Note that this tooltip style sets the style for all tool_tips.
    # background-color: #F0EFEF;border: 2px solid black;font-family: arial; padding: 10px; background-color: #F0EFEF;
)

tooltip_seg = folium.GeoJsonTooltip(
    fields=["nhm_seg", "tosegment_nhm"],
    aliases=["segment", "flows to segment"],
    labels=True,
)
popup_seg = folium.GeoJsonPopup(
    fields=["nhm_seg", "tosegment_nhm"],
    aliases=["segment", "flows to segment"],
    labels=True,
    localize=False,
)

# Create and add HRU and Segment maps
# hru_gdf[['nhm_id','hru_segment_nhm']] = hru_gdf[['nhm_id','hru_segment_nhm']].astype(object)
hru_map = folium.GeoJson(
    hru_gdf,
    style_function=style_function_hru_map,
    highlight_function=highlight_function_hru_map,
    name="NHM HRUs",
    # tooltip=tooltip_hru,
    popup=popup_hru,
).add_to(m2)

seg_map_show = folium.GeoJson(
    seg_gdf,
    style_function=style_function_seg_map,
    highlight_function=highlight_function_seg_map,  # lambda feature: {"fillcolor": "white", "color": "white"},
    name="NHM Segments",
).add_to(m2)

# This makes a transparent layer that will always be on the top and so the tooltip is associated with this layer, and will also always be on top.
seg_map_hide = folium.GeoJson(
    seg_gdf,
    style_function=transparent,
    highlight_function=highlight_function_seg_map,  # lambda feature: {"fillcolor": "white", "color": "white"},
    name="NHM Segments",
    # tooltip=tooltip_seg,
    popup=popup_seg,
).add_to(m2)

# add POI marker cluster child items for the map
marker_cluster = MarkerCluster(
    name="All the POIs",
    overlay=True,
    control=True,
    icon_create_function=None,
    disableClusteringAtZoom=7,
)
marker_cluster_label = MarkerCluster(
    name="All the POIs labels",
    overlay=True,
    control=True,
    show=False,  # False will not draw the child upon opening the map, but have it to draw in the Layer control.
    icon_create_function=None,
    disableClusteringAtZoom=7,
)

# add non-poi gages marker cluster child items for the map
non_marker_cluster = MarkerCluster(
    name="Non-poi gages",
    overlay=True,
    control=True,
    icon_create_function=None,
    disableClusteringAtZoom=7,
)
non_marker_cluster_label = MarkerCluster(
    name="Non-poi gage labels",
    overlay=True,
    control=True,
    show=False,  # False will not draw the child upon opening the map, but have it to draw in the Layer control.
    icon_create_function=None,
    disableClusteringAtZoom=7,
)


##add POI markers and labels using row df.interowss loop
for idx, row in poi_gdf.iterrows():
    text = f'{row["poi_id"]}'
    label_lat = row["latitude"]  # -0.01
    label_lon = row["longitude"]

    marker_label = folium.map.Marker(
        [label_lat, label_lon],
        icon=DivIcon(
            icon_size=(10, 10),  # (150,36),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12pt; font-weight: bold">%s</div>' % text,
        ),
    ).add_to(marker_cluster_label)

    marker = folium.CircleMarker(
        location=[row["latitude"], row["longitude"]],
        name=row["poi_id"],
        popup=folium.Popup(
            f'<font size="3px">{row["poi_id"]} ({row["poi_agency"]})<br>{row["poi_name"]}<br> on <b>segment </b>{row["nhm_seg"]}</font>',
            max_width=280,
            max_height=2000,
        ),
        radius=5,
        weight=2,
        color="black",
        fill=True,
        fill_color="Yellow",
        fill_opacity=1.0,
    ).add_to(marker_cluster)

# Add tool tips to map
# hru_map.add_child(tooltip_hru)
# seg_map_hide.add_child(tooltip_seg)

marker_cluster.add_to(m2)
marker_cluster_label.add_to(m2)

##add Non-poi gage markers and labels using row df.interowss loop
for idx, row in nwis_gages_aoi.iterrows():
    if row["poi_id"] in additional_gages:

        text = f'{row["poi_id"]}'
        label_lat = row["latitude"]  # -0.01
        label_lon = row["longitude"]

        marker_label = folium.map.Marker(
            [label_lat, label_lon],
            icon=DivIcon(
                icon_size=(10, 10),  # (150,36),
                icon_anchor=(0, 0),
                html='<div style="font-size: 12pt; font-weight: bold">%s</div>' % text,
            ),
        ).add_to(non_marker_cluster_label)

        marker = folium.CircleMarker(
            location=[row["latitude"], row["longitude"]],
            name=row["poi_id"],
            popup=folium.Popup(
                f'<font size="3px">{row["poi_id"]} ({row["poi_agency"]})<br>{row["poi_name"]}<br></font>',
                max_width=280,
                max_height=2000,
            ),
            radius=5,
            weight=2,
            color="black",
            fill=True,
            fill_color="Red",
            fill_opacity=1.0,
        ).add_to(non_marker_cluster)
    else:
        pass

# Add tool tips to map
# hru_map.add_child(tooltip_hru)
# seg_map_hide.add_child(tooltip_seg)

non_marker_cluster.add_to(m2)
non_marker_cluster_label.add_to(m2)

m2.add_child(minimap)
plugins.Fullscreen(position="bottomleft").add_to(m2)
folium.LayerControl(collapsed=True, position="bottomright").add_to(m2)

con.print(f"")
con.print(f"")
con.print(f"")
con.print(f"The NHMx framework:", style="u bold black")
con.print(
    f"{pdb.dimensions.get('nhru').meta['description']} (nhru), {pdb.dimensions.get('nhru').meta['size']}"
)
con.print(
    f"{pdb.dimensions.get('nsegment').meta['description']} (nsegment), {pdb.dimensions.get('nsegment').meta['size']}"
)
con.print(
    f"{pdb.dimensions.get('npoigages').meta['description']} (npoigages), {pdb.dimensions.get('npoigages').meta['size']}"
)

m2